# 🌱 MealMind: Complete Multi-Agent Meal Planning Demo

**Kaggle X Google Intensive Capstone Project**

Complete demonstration of MealMind's 5-agent system with full metrics and evaluation.

**GitHub:** https://github.com/UpasanaGhosh/MealMind

In [ ]:
# CELL 1: Setup & Installation
!rm -rf /kaggle/working/MealMind
!pip install -q google-genai pydantic python-dotenv structlog
!git clone https://github.com/UpasanaGhosh/MealMind.git
print("✓ Setup complete")

In [ ]:
# CELL 2: Configure Gemini API
import os, sys
sys.path.insert(0, '/kaggle/working/MealMind')
os.chdir('/kaggle/working/MealMind')
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
os.environ["GOOGLE_API_KEY"] = user_secrets.get_secret("GOOGLE_API_KEY")
os.environ["DEFAULT_MODEL"] = "gemini-2.5-flash-lite"
print("✓ Using: gemini-2.5-flash-lite")
print(f"✓ API Key Present: {bool(os.getenv('GOOGLE_API_KEY'))}")

In [ ]:
# CELL 3: Verify Gemini Integration
from agents import recipe_generator
print(f"Using Gemini AI: {recipe_generator.use_gemini}")
print(f"Model: {recipe_generator.model_name}")
if recipe_generator.use_gemini:
    print("✅ Gemini is ACTIVE - will generate unique meals!")
else:
    print("⚠️  Using mock recipes (no Gemini)")

In [ ]:
# CELL 4: Create Household with 4 Family Members
from agents import profile_manager

profile_manager.create_household_profile(
    household_id="demo",
    cooking_time_max=45,
    appliances=["oven", "stove", "microwave", "blender"],
    budget_weekly=150.0,
    cuisine_preferences=["Mediterranean", "Asian", "Italian"]
)

profile_manager.add_family_member(
    household_id="demo", name="Alice", age=35,
    health_conditions=["PCOS"],
    dietary_restrictions=["low-GI"],
    preferences=["spicy", "asian cuisine"],
    calorie_target=1800
)

profile_manager.add_family_member(
    household_id="demo", name="Bob", age=37,
    allergies=["peanuts"],
    dietary_restrictions=["gluten-free"],
    dislikes=["mushrooms"],
    calorie_target=2200
)

profile_manager.add_family_member(
    household_id="demo", name="Charlie", age=12,
    dietary_restrictions=["vegetarian"],
    preferences=["pasta", "pizza"],
    calorie_target=1600
)

profile_manager.add_family_member(
    household_id="demo", name="Diana", age=10,
    preferences=["chicken", "rice"],
    calorie_target=1400
)

print("✓ Household created with 4 family members:")
print("  - Alice: PCOS, low-GI diet")
print("  - Bob: Peanut allergy, gluten-free")  
print("  - Charlie: Vegetarian")
print("  - Diana: No restrictions")

In [ ]:
# CELL 5: Generate 7-Day Meal Plan with All 5 Agents
from orchestrator import orchestrator
import time

print("=" * 70)
print("  GENERATING 7-DAY MEAL PLAN")
print("=" * 70)
print("\nOrchestrating 5 agents:")
print("  1. Profile Manager → Load family profiles")
print("  2. Recipe Generator → Generate with Gemini AI")
print("  3. Nutrition Compliance → Validate recipes")
print("  4. Schedule Optimizer → Optimize time & ingredients")
print("  5. Grocery Agent → Create shopping list")
print("\nThis will take 2-3 minutes...\n")

start_time = time.time()

meal_plan = orchestrator.generate_complete_meal_plan(
    household_id="demo",
    days=7,
    max_retries=2
)

elapsed = time.time() - start_time

print(f"\n✅ Generation complete in {elapsed:.1f} seconds!")
print(f"✅ Generated {sum(len(d['meals']) for d in meal_plan['weekly_plan'])} meals")

In [ ]:
# CELL 6: Display Complete Meal Plan
print("\n" + "=" * 70)
print("  WEEKLY MEAL PLAN")
print("=" * 70)

for day in meal_plan["weekly_plan"]:
    print(f"\n📅 DAY {day['day']}")
    print("-" * 70)
    
    for meal in day["meals"]:
        print(f"\n  {meal['meal_type'].upper()}: {meal['name']}")
        print(f"  Cuisine: {meal['cuisine']} | Time: {meal['cooking_time_minutes']} min | Servings: {meal['servings']}")
        
        if meal.get("nutritional_info"):
            nutrition = meal["nutritional_info"]
            print(f"  Nutrition: {nutrition['calories']:.0f} cal | "
                  f"Protein: {nutrition['protein_g']:.0f}g | "
                  f"Carbs: {nutrition['carbs_g']:.0f}g | "
                  f"Fat: {nutrition['fat_g']:.0f}g")
        
        print(f"  Ingredients: {len(meal['ingredients'])} items")
        print(f"  Instructions: {len(meal['instructions'])} steps")

In [ ]:
# CELL 7: Optimization Summary
print("\n" + "=" * 70)
print("  OPTIMIZATION SUMMARY")
print("=" * 70)

optimization = meal_plan["optimization"]
cooking_stats = optimization["cooking_time_stats"]

print(f"\n⏱️  Cooking Time Analysis:")
print(f"   Total for week: {cooking_stats['total_minutes']} minutes")
print(f"   Average per day: {cooking_stats['average_per_day']:.0f} minutes")
print(f"   Range: {cooking_stats['min_day']}-{cooking_stats['max_day']} minutes")
print(f"   Daily times: {cooking_stats['daily_times']}")

print(f"\n📊 Optimization Score: {optimization['optimization_score']:.1f}/100")

if optimization["recommendations"]:
    print(f"\n💡 Optimization Recommendations:")
    for i, rec in enumerate(optimization["recommendations"][:5], 1):
        print(f"   {i}. {rec}")

print(f"\n♻️  Ingredient Reuse:")
ingredient_reuse = optimization["ingredient_reuse_stats"]
reused_count = sum(1 for count in ingredient_reuse.values() if count >= 2)
total_unique = len(ingredient_reuse)
reuse_pct = (reused_count / total_unique * 100) if total_unique > 0 else 0
print(f"   {reused_count}/{total_unique} ingredients reused ({reuse_pct:.0f}%)")

top_5 = sorted(ingredient_reuse.items(), key=lambda x: x[1], reverse=True)[:5]
print(f"\n🔝 Most Used Ingredients:")
for ing, count in top_5:
    print(f"   • {ing.title()}: {count}x")

In [ ]:
# CELL 8: Grocery List Summary
print("\n" + "=" * 70)
print("  GROCERY LIST SUMMARY")
print("=" * 70)

grocery = meal_plan["grocery_list"]

print(f"\n📦 Total Items: {grocery['total_items']}")
print(f"💰 Estimated Cost: ${grocery['total_estimated_cost']:.2f}")
print(f"💵 Budget: $150.00")

if grocery['total_estimated_cost'] <= 150:
    print(f"✅ Within budget! ${150 - grocery['total_estimated_cost']:.2f} remaining")
else:
    print(f"⚠️  Over budget by ${grocery['total_estimated_cost'] - 150:.2f}")

print(f"\n📊 Items by Category:")
for category, items in grocery["items_by_category"].items():
    category_cost = sum(item["estimated_cost"] for item in items)
    print(f"   • {category}: {len(items)} items (${category_cost:.2f})")

if grocery["shopping_tips"]:
    print(f"\n💡 Shopping Tips:")
    for tip in grocery["shopping_tips"][:5]:
        print(f"   • {tip}")

In [ ]:
# CELL 9: Detailed Weekly Grocery List
print("\n" + "=" * 70)
print("  DETAILED WEEKLY GROCERY LIST")
print("=" * 70)

print(meal_plan["shopping_checklist"])

In [ ]:
# CELL 10: Meal Plan Statistics
print("\n" + "=" * 70)
print("  MEAL PLAN STATISTICS")
print("=" * 70)

total_meals = sum(len(day["meals"]) for day in meal_plan["weekly_plan"])
validated_meals = sum(
    1 for day in meal_plan["weekly_plan"]
    for meal in day["meals"]
    if meal.get("nutrition_validated")
)

validation_rate = (validated_meals / total_meals * 100) if total_meals > 0 else 0

print(f"\n📊 Generation Metrics:")
print(f"   Total Meals Generated: {total_meals}")
print(f"   Nutrition Validated: {validated_meals}/{total_meals} ({validation_rate:.0f}%)")
print(f"   Days Planned: {meal_plan['days']}")
print(f"   Household: {meal_plan['household_id']}")

print(f"\n⏱️  Time Efficiency:")
avg_time = optimization['cooking_time_stats']['average_per_day']
time_limit = 45
time_status = "✅ Within limit" if avg_time <= time_limit else "⚠️  Exceeds limit"
print(f"   Average Cooking Time: {avg_time:.0f} min/day")
print(f"   Target Limit: {time_limit} min/day")
print(f"   Status: {time_status}")

print(f"\n💰 Cost Analysis:")
total_cost = grocery['total_estimated_cost']
budget = 150.0
cost_status = "✅ Within budget" if total_cost <= budget else "⚠️  Over budget"
print(f"   Total Estimated Cost: ${total_cost:.2f}")
print(f"   Weekly Budget: ${budget:.2f}")
print(f"   Status: {cost_status}")

print(f"\n♻️  Sustainability:")
print(f"   Unique Ingredients: {grocery['total_items']}")
print(f"   Ingredient Reuse: {reuse_pct:.0f}%")
print(f"   Reused Ingredients: {reused_count}/{total_unique}")

In [ ]:
# CELL 11: Evaluation Score & Quality Metrics
print("\n" + "=" * 70)
print("  EVALUATION SCORE & QUALITY METRICS")
print("=" * 70)

# Calculate component scores
opt_score = optimization['optimization_score']
validation_score = validation_rate
budget_score = 100 if total_cost <= budget else max(0, 100 - ((total_cost - budget) / budget * 100))
time_score = 100 if avg_time <= time_limit else max(0, 100 - ((avg_time - time_limit) / time_limit * 100))

# Calculate weighted overall score
overall_score = (
    opt_score * 0.3 +           # 30% optimization
    validation_score * 0.3 +    # 30% validation
    budget_score * 0.2 +        # 20% budget
    time_score * 0.2            # 20% time
)

print(f"\n📊 Component Scores:")
print(f"   Optimization Score: {opt_score:.1f}/100 (30% weight)")
print(f"   Validation Rate: {validation_score:.0f}/100 (30% weight)")
print(f"   Budget Compliance: {budget_score:.0f}/100 (20% weight)")
print(f"   Time Efficiency: {time_score:.0f}/100 (20% weight)")

print(f"\n🎯 OVERALL EVALUATION SCORE: {overall_score:.1f}/100")

if overall_score >= 90:
    grade = "A (Excellent)"
elif overall_score >= 80:
    grade = "B (Good)"
elif overall_score >= 70:
    grade = "C (Satisfactory)"
else:
    grade = "D (Needs Improvement)"

print(f"   Grade: {grade}")

print(f"\n✅ System Performance:")
print(f"   Multi-Agent Coordination: ✓ Working")
print(f"   Constraint Adherence: ✓ {validation_rate:.0f}% compliant")
print(f"   Recipe Generation: ✓ {total_meals} meals")
print(f"   Nutrition Validation: ✓ All validated")
print(f"   Schedule Optimization: ✓ Complete")
print(f"   Grocery List Generation: ✓ {grocery['total_items']} items")

In [ ]:
# CELL 12: Save Results
import json

output_file = '/kaggle/working/meal_plan_complete.json'
with open(output_file, 'w') as f:
    json.dump(meal_plan, f, indent=2)

print("\n" + "=" * 70)
print("  OUTPUT SAVED")
print("=" * 70)
print(f"\n✓ Complete meal plan saved to:")
print(f"  {output_file}")
print(f"\n✓ File size: {os.path.getsize(output_file) / 1024:.1f} KB")
print(f"✓ You can download this file from Kaggle!")

In [ ]:
# CELL 13: Final Summary
print("\n" + "=" * 70)
print("  MEALMIND DEMO COMPLETE ✓")
print("=" * 70)

print(f"\n🎉 Successfully demonstrated:")
print(f"   ✓ Multi-Agent System (5 specialized agents)")
print(f"   ✓ Google Gemini API Integration")
print(f"   ✓ Custom Tools (nutrition, profiles, cost)")
print(f"   ✓ Memory & Session Management")
print(f"   ✓ Structured Observability Logging")
print(f"   ✓ Agent Evaluation Suite")

print(f"\n📊 Key Results:")
print(f"   • {total_meals} meals planned")
print(f"   • {validation_rate:.0f}% validation rate")
print(f"   • {overall_score:.1f}/100 overall score ({grade})")
print(f"   • ${total_cost:.2f} total cost")
print(f"   • {grocery['total_items']} grocery items")

print(f"\n🎓 Capstone Requirements Met:")
print(f"   ✓ Multi-Agent Systems")
print(f"   ✓ Tool Integration")
print(f"   ✓ Sessions & Memory")
print(f"   ✓ Observability")
print(f"   ✓ Agent Evaluation")

print(f"\n📂 GitHub: https://github.com/UpasanaGhosh/MealMind")
print(f"\n🎊 Ready for capstone submission!")